<a href="https://www.kaggle.com/code/sharabhojha/chord-generation-lstm-example?scriptVersionId=222908066" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tinyaam/audio-mixes-mp3/0620_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/2841_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/2462_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/0989_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/0758_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/3000_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/0080_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/1014_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/2990_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/2395_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/0001_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/2720_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/1050_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/1545_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/2602_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/0192_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/2828_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/1941_mix.mp3
/kaggle/input/tinyaam/audio-mixes-mp3/2269_mix.mp3
/kaggle/input/tinyaam/audio-mix

In [2]:
# Preprocessing steps:
# 1: condense onset files to just onset times and onset notes

import os
import re
import pandas as pd

# Directory path to your annotations folder
annotations_dir = "/kaggle/input/tinyaam/annotations/"

# Iterate through all files in the directory
for filename in os.listdir(annotations_dir):
    if "onsets" in filename and filename.endswith(".arff"):  # Ensure it's an ARFF file with 'onsets' in its name
        file_path = os.path.join(annotations_dir, filename)
        
        # Read the ARFF file
        with open(file_path, "r", encoding="utf-8") as file:
            lines = file.readlines()

        attributes = []
        data_start = False
        data_rows = []

        for line in lines:
            line = line.strip()
            
            # Ignore comments and empty lines
            if not line or line.startswith("%"):
                continue

            if line.lower().startswith("@attribute"):
                # Extract attribute name (between the first space and the last space)
                match = re.match(r"@attribute\s+['\"]?([\w\s]+)['\"]?\s+.*", line, re.IGNORECASE)
                if match:
                    attributes.append(match.group(1).strip())

            elif line.lower().startswith("@data"):
                data_start = True  # Data section starts
            
            elif data_start:
                # Split considering quoted strings properly
                values = re.findall(r'\".*?\"|\'.*?\'|[^,]+', line)
                values = [v.strip("\"' ") if v.strip() else None for v in values]  # Remove extra quotes
                data_rows.append(values)

        # Convert to DataFrame
        df = pd.DataFrame(data_rows, columns=attributes)

        # Convert numeric columns where possible
        for col in df.columns:
            try:
                df[col] = pd.to_numeric(df[col])  # Convert if possible
            except ValueError:
                pass  # Keep as string if conversion fails

        all_onsets = []

        # collect all onset events per timestamp
        for row in range(df.index.size):
            notes_at_onset = []
            for col in range(1, df.columns.size):
                notes_at_onset.append(df.iat[row, col])
            notes_at_onset = re.findall("(\d+)", ''.join(notes_at_onset))
            all_onsets.append([int(item) for item in notes_at_onset])

        # delete onset events for individual instruments and add column for all events
        allCols = df.columns[df.apply(lambda col: col.astype(str).str.contains(r"\[", regex=True)).any()].tolist()
        df.drop(allCols, axis=1, inplace=True)
        df["Onset events"] = all_onsets

        # Save the processed DataFrame to a new CSV file
        output_file = re.search("(\d+)", filename).group(0) + "_onset_condensed.csv"
        df.to_csv(output_file, index=False)

        print(f"Processed {filename} and saved to {output_file}")

Processed 2990_onsets.arff and saved to 2990_onset_condensed.csv
Processed 0989_onsets.arff and saved to 0989_onset_condensed.csv
Processed 0758_onsets.arff and saved to 0758_onset_condensed.csv
Processed 2602_onsets.arff and saved to 2602_onset_condensed.csv
Processed 1050_onsets.arff and saved to 1050_onset_condensed.csv
Processed 0192_onsets.arff and saved to 0192_onset_condensed.csv
Processed 1014_onsets.arff and saved to 1014_onset_condensed.csv
Processed 1941_onsets.arff and saved to 1941_onset_condensed.csv
Processed 2841_onsets.arff and saved to 2841_onset_condensed.csv
Processed 0001_onsets.arff and saved to 0001_onset_condensed.csv
Processed 0080_onsets.arff and saved to 0080_onset_condensed.csv
Processed 2462_onsets.arff and saved to 2462_onset_condensed.csv
Processed 2828_onsets.arff and saved to 2828_onset_condensed.csv
Processed 1545_onsets.arff and saved to 1545_onset_condensed.csv
Processed 2269_onsets.arff and saved to 2269_onset_condensed.csv
Processed 1711_onsets.arf

In [3]:
# 2: encode chord names and replace said chord names with encodings in beatinfo files

# Directory path to your annotations folder
annotations_dir = "/kaggle/input/tinyaam/annotations/"
headers = ['Start time in seconds', 'Bar count', 'Quarter count', 'Chord name']

chords = set()

dataframes = []
filenames = []

# Iterate through all files in the directory
for filename in os.listdir(annotations_dir):
    if "beatinfo" in filename and filename.endswith(".arff"):  # Ensure it's an ARFF file with 'beatinfo' in its name
        file_path = os.path.join(annotations_dir, filename)
        filenames.append(filename)
        df = pd.read_csv(file_path, comment='@', header=None)
        df.columns = headers

        for i in range(df.index.size):
            df.iat[i, 3] = df.iat[i, 3].replace("'", "")
            chords.add(df.iat[i, 3])
    
        dataframes.append(df)

sorted_chords = sorted(list(chords))
chord_encodings = dict(zip([i for i in range(len(chords))], sorted_chords))
print(chord_encodings)

# Modify existing dataframes to match encodings
for i in range(len(dataframes)):
    dataframes[i].drop(columns=['Bar count', 'Quarter count'], inplace=True)

    for j in range(dataframes[i].index.size):
        dataframes[i].iat[j, 1] = sorted_chords.index(dataframes[i].iat[j, 1])

    dataframes[i].to_csv(filenames[i].replace('arff', 'csv'), index=False)

{0: 'A#maj', 1: 'A#min', 2: 'Amaj', 3: 'Amin', 4: 'Bmaj', 5: 'Bmin', 6: 'C#maj', 7: 'C#min', 8: 'Cmaj', 9: 'Cmin', 10: 'D#maj', 11: 'D#min', 12: 'Dmaj', 13: 'Dmin', 14: 'Emaj', 15: 'Emin', 16: 'F#maj', 17: 'F#min', 18: 'Fmaj', 19: 'Fmin', 20: 'G#maj', 21: 'G#min', 22: 'Gmaj', 23: 'Gmin', 24: 'N.C.'}


In [28]:
# visualize the files
working_dir = "/kaggle/working/"
onsets = pd.read_csv(working_dir + "0001_onset_condensed.csv")
print(onsets.head())
beatinfo = pd.read_csv(working_dir + "0001_beatinfo.csv")
print(beatinfo.head())

def align_onsets_with_chords(onsets, beatinfo):
    aligned_data = []
    for _, onset_row in onsets.iterrows():
        onset_time = onset_row['Onset time in seconds']
        # Find the chord corresponding to this onset time
        chord_row = beatinfo[beatinfo['Start time in seconds'] <= onset_time].iloc[-1]
        onset_list = eval(onset_row['Onset events'])
        aligned_data.append((onset_list, chord_row['Chord name']))
    return aligned_data

   Onset time in seconds  Onset events
0               0.000000  [41, 60, 65]
1               0.326086      [41, 60]
2               0.652173  [41, 65, 65]
3               0.978259  [41, 65, 69]
4               1.304346  [41, 65, 65]
   Start time in seconds  Chord name
0               0.000000          18
1               0.652174          18
2               1.304348          18
3               1.956522          18
4               2.608696           0


In [34]:
# create aligned data for every onset and beatinfo file

all_data = []
working_dir = "/kaggle/working/"

for filename in os.listdir(working_dir):
    if "onset" in filename:
        onset_path = os.path.join(working_dir, filename)
        beatinfo_path = os.path.join(working_dir, re.search("(\d+)", filename).group(0) + "_beatinfo.csv")
        onsets = pd.read_csv(onset_path)
        beatinfo = pd.read_csv(beatinfo_path)
        all_data += align_onsets_with_chords(onsets, beatinfo)

#for i in range(len(all_data)):
    #all_data[i][0][0] = eval(all_data[i][0][0])
    

print(all_data)

[([55, 58, 63, 36, 42, 39, 70, 79], 10.0), ([55, 58, 63, 42, 39, 63, 79], 10.0), ([55, 58, 63, 38, 42, 39, 67, 77], 10.0), ([55, 58, 63, 42, 39, 63, 77], 10.0), ([55, 58, 63, 42, 39, 70, 74], 10.0), ([55, 58, 63, 36, 42, 39, 67, 74], 10.0), ([55, 58, 63, 38, 42, 39, 70, 72], 10.0), ([55, 58, 63, 36, 42, 39, 63, 72], 10.0), ([55, 58, 63, 36, 42, 39, 70, 77], 10.0), ([55, 58, 63, 42, 39, 63, 75], 10.0), ([55, 58, 63, 38, 42, 39, 67, 75], 10.0), ([55, 58, 63, 42, 39, 63], 10.0), ([55, 58, 63, 42, 39, 70, 74], 10.0), ([55, 58, 63, 36, 42, 39, 67], 10.0), ([55, 58, 63, 38, 42, 39, 70, 75], 10.0), ([55, 58, 63, 36, 42, 39, 63, 75], 10.0), ([55, 58, 62, 36, 42, 43, 62, 79], 10.0), ([55, 58, 62, 42, 43, 67, 79], 23.0), ([55, 58, 62, 38, 42, 43, 70, 74], 23.0), ([55, 58, 62, 42, 43, 67], 23.0), ([55, 58, 62, 42, 43, 62, 72], 23.0), ([55, 58, 62, 36, 42, 43, 70, 79], 23.0), ([55, 58, 62, 38, 42, 43, 62, 74], 23.0), ([55, 58, 62, 36, 42, 43, 67, 74], 23.0), ([55, 58, 63, 36, 42, 39, 70, 72], 23.0

In [33]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Masking
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Convert string lists to actual lists
X_raw = [x[0] for x in all_data]  # Convert string lists to actual lists
y_raw = np.array([x[1] for x in all_data])  # Target values

# Normalize target values
scaler = MinMaxScaler()
y_scaled = scaler.fit_transform(y_raw.reshape(-1, 1))

# Pad sequences to make them equal length
X_padded = pad_sequences(X_raw, padding='post', dtype='float32')

# Reshape for LSTM (samples, timesteps, features)
X_final = np.expand_dims(X_padded, axis=-1)

# Split into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X_final, y_scaled, test_size=0.2, random_state=42)

# Build LSTM Model
model = Sequential([
    Masking(mask_value=0.0, input_shape=(X_final.shape[1], 1)),  # Ignore padded zeros
    LSTM(50, return_sequences=True),
    LSTM(50),
    Dense(50, activation='relu'),
    Dense(25, activation='softmax')  # 25-class classification
])

model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=4, validation_data=(X_test, y_test))

# Predict on test set
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)  # Convert back to original scale

print("Predicted Values:", predictions.flatten())

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/masking.py:47: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


2255/2255 ━━━━━━━━━━━━━━━━━━━━ 20s 7ms/step - loss: 0.2881 - mae: 0.4449 - val_loss: 0.2859 - val_mae: 0.4469
Epoch 2/10
2255/2255 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.2840 - mae: 0.4408 - val_loss: 0.2859 - val_mae: 0.4469
Epoch 3/10
2255/2255 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.2886 - mae: 0.4457 - val_loss: 0.2859 - val_mae: 0.4469
Epoch 4/10
2255/2255 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.2889 - mae: 0.4463 - val_loss: 0.2859 - val_mae: 0.4469
Epoch 5/10
2255/2255 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.2904 - mae: 0.4460 - val_loss: 0.2859 - val_mae: 0.4469
Epoch 6/10
2255/2255 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.2843 - mae: 0.4413 - val_loss: 0.2859 - val_mae: 0.4469
Epoch 7/10
2255/2255 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.2853 - mae: 0.4415 - val_loss: 0.2859 - val_mae: 0.4469
Epoch 8/10
2255/2255 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - loss: 0.2834 - mae: 0.4398 - val_loss: 0.2859 - val_mae: 0.4469
Epoch 9/10
2255/2255 ━━━━━━━━━━━━━━━━━━━━ 1